In [ ]:
#fear emotion

#데이터 pandas로 읽기, 구문이 탭으로 구분되어있음.
import pandas as pd
train_fear_df = pd.read_csv("feartrainalldata.txt","\t") 
test_fear_df = pd.read_csv("feartestalldata.txt","\t")

#konlpy는 띄어쓰기 알고리즘과 정규화를 이용해 맞춤법이 틀린 문장을 고치고 형태소 분석과 품사를 태깅해주는 클래스를 제공함.
from konlpy.tag import Okt
okt_fear = Okt()
# Okt를 쓴 이유 : 사용한 데이터는 인터넷 댓글이다. 어느정도 띄어쓰기가 되어있는 상태이기에 Okt가 가장 유리하다.

def fear_tokenize(doc):
    return ['/'.join(t) for t in okt_fear.pos(doc, norm=True, stem=False)]#08.22 stem=false로 바꿔봄. 
    #'구분자'.join(list) => 리스트 값과 값 사이에 구분자를 넣어 문자열을 하나로 합침. ex) '_'.join(['a','b','c'])이면 "a_b_c"로 반환
    #pos(text) => 품사를 태깅한 상태로 명사를 변환, ex) 컴퓨터(x), 컵퓨터,noun(o)
    #norm => 정규화 ex) 그래욬ㅋㅋㅋ => 그래요
    #stem => 근어로 표현 ex) 그래요 => 그렇다

train_fear_df.isnull().any()
train_fear_df['document'] = train_fear_df['document'].fillna(''); #null값을 ''로 대체
test_fear_df.isnull().any()
test_fear_df['document'] = test_fear_df['document'].fillna('');

train_fear_docs = [(fear_tokenize(row[1]), row[2]) for row in train_fear_df.values] #traindata 저장 row[1] : document / row[2] : label
test_fear_docs = [(fear_tokenize(row[1]),row[2]) for row in test_fear_df.values] #testdata 저장

tokens = [t for d in train_fear_docs for t in d[0]]

import nltk
fear_text = nltk.Text(tokens, name='FEAR') 
#문서 하나를 편리하게 탐색할 수 있는 기능 제공 (vocab().most_common() 사용)
# print(len(fear_text.tokens)) #토큰 개수
# print(len(set(fear_text.tokens))) #중복을 제외한 토큰 수
# print(fear_text.vocab().most_common(10)) #출력빈도가 높은 상위 토큰 10개

#countvectorization
FEAR_FREQUENCY_COUNT = 3000; #자주 사용되는 토큰 설정 3000이상 하면 메모리 에러가 발생한다.
fear_selected_words = [f[0] for f in fear_text.vocab().most_common(FEAR_FREQUENCY_COUNT)] #선택되어진 토큰들

def fear_term_frequency(doc):
    return [doc.count(word) for word in fear_selected_words]

x_fear_train = [fear_term_frequency(d) for d,_ in train_fear_docs]
x_fear_test = [fear_term_frequency(d) for d,_ in test_fear_docs]
#x축에는 문서에 들어가는 단어 개수(단어들의 빈도수 정보)

y_fear_train = [c for _,c in train_fear_docs]
y_fear_test = [c for _,c in test_fear_docs]
#y축에는 1 or 0, 분류 결과

import numpy as np
x_fear_train = np.asarray(x_fear_train).astype('float32')
x_fear_test = np.asarray(x_fear_test). astype('float32')

y_fear_train = np.asarray(y_fear_train).astype('float32')
y_fear_test = np.asarray(y_fear_test).astype('float32')
#np.asarray는 np.array와 달리 데이터 형태가 같을 때 복사하지 않음.
#데이터 float로 형 변환

import tensorflow as tf #텐서플로 케라스
fear_model= tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation = 'relu',input_shape=(FEAR_FREQUENCY_COUNT,)),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])
#레이어 구성은 dense층은 64개의 유닛, 활성함수는 relu, 마지막 층은 sigmoid 활성화 함수 사용
# relu를 쓴 이유 : 연산의 cost가 적고, 구현이 간단함.
# sigmoid를 쓴 이유 : 다중 감정 분석이지만 이진 분류 문제로 해결했기 때문에 출력층의 활성화함수는 이진 분류 문제에 효과적인 시그모이드 사용 
# relu층이 두 층인 이유 : 세 층으로 했을 때, testset에 대한 정확도가 떨어졌다. 단순히 하는 것이 overfitting을 막는 방법
# dense층이 64개의 유닛인 이유는?

fear_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001),
             loss=tf.keras.losses.binary_crossentropy,
             metrics=[tf.keras.metrics.binary_accuracy]
             )
#손실 함수는 binary_crossentropy, RMSprop 옵티마이저를 통해 경사하강법 진행
#binary_crossentropy를 사용한 이유는 가장 이진 분류에 가장 적합하기 때문이다.
#RMSprop와 Adam은 거의 비슷하다. Adam을 쓰면 메모리 에러가 발생함.

fear_model.fit(x_fear_train, y_fear_train, epochs=50, batch_size=256)
#배치 사이즈 줄이면 한 번에 판단하는 데이터 수 증가함. 에포크 50번
#에포크에 따른 손실함수 그래프를 통해 50번이 적당하다고 판단하였다.

fear_results = fear_model.evaluate(x_fear_test, y_fear_test)

fear_review = "너무 무섭다"
fear_token = fear_tokenize(fear_review)

fear_tf = fear_term_frequency(fear_token)
f_data = np.expand_dims(np.asarray(fear_tf).astype('float32'),axis=0)
float(fear_model.predict(f_data))